<a href="https://colab.research.google.com/github/richmondvan/ringview/blob/master/RingviewTrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### Imports

In [ ]:
from google.colab.patches import cv2_imshow # For image previews
import numpy as np # for linear algebra help
from math import inf # for math help :D
from random import randint, random # for random number generation
from skimage.metrics import structural_similarity as ss # For similarity measurement
import cv2 # opencv2 for image management

from PIL import Image, ImageFilter # for initial image palette and bgcolor generation with help of colorthief
%pip install colorthief
from colorthief import ColorThief # For grabbing color information from source image

### Create some helper methods

In [ ]:
def changeColorFromRGBToBGR(color): # converts from RGB (colorthief colors) to BGR format (opencv2 colors)
    return (color[2], color[1], color[0])

def showImage(image): # Shows the image, this is placeholder for VSCode to Colab conversions (since cv2_imshow does not work outside of colab)
    cv2_imshow(image)

# # Mean squared error function, can be reused later
# def calculateError(image, source):
#     return np.square(image - source).mean()

# Structural simularity error function
def calculateError(image, source):
    return -1 * ss(image, source, multichannel=True) # must be negative because simularity is positive but error is negative

def addCircle(image, gene): # Adds a circle based on gene information onto an image using opencv2 blending modes.
    overlay = image.copy()
    cv2.circle(
        overlay,
        center=gene.center,
        radius=gene.radius,
        color=palette[gene.color],
        thickness=-1
    )
    cv2.addWeighted(overlay, gene.alpha, image, 1-gene.alpha, 0, image)

### Load an example into memory

In [ ]:
# Trial filenames
BUTTERFLY_FILENAME = "img.jpg"
MONALISA_FILENAME = "monalisa224.png"
MONALISASQUARE_FILENAME = "monalisasquare.png" # Use this!

TEMP = "temp.jpg" # Do not overwrite original image with touchups

NUM_COLORS = 32 # Number of colors in color palette (this is approximate, colorthief is inconsistent)

BASEIMAGE_FILENAME = MONALISASQUARE_FILENAME # Set this to filename of proper image

# Smooth image twice and save it in temporary file (do not overwrite original!)
baseImage = Image.open(BASEIMAGE_FILENAME) 
smoothenedImage = baseImage.filter(ImageFilter.SMOOTH_MORE)
smoothenedImage = smoothenedImage.filter(ImageFilter.SMOOTH_MORE)
smoothenedImage.save(TEMP)

# generate some colours out of smoothened image, remember to convert from RGB to BGR
colorThief = ColorThief(TEMP)
backgroundColor = changeColorFromRGBToBGR(colorThief.get_color(quality=1)) # gets the background color
palette = colorThief.get_palette(color_count=NUM_COLORS, quality=1) # gets the top colors used by the image.
for index, color in enumerate(palette):
    palette[index] = changeColorFromRGBToBGR(color)

BGRbaseImage =  cv2.imread(TEMP)
showImage(BGRbaseImage) # Preview image

### Prepare for evolution

In [ ]:
# Set evolution constants

NUM_GENERATIONS = 20000 # 20000 default. Number of generations to run.
NUM_CIRCLES = 1000 # 1000 default. Number of circles to draw. 

# Calculations for minimum and maximum radius to draw on images.
MINIMUM_RADIUS = min(20, int(0.04 * min(baseImage.size)))
MAXIMUM_RADIUS = max(40, int(0.08 * max(baseImage.size)))

# width and height variables for convenience.
width = baseImage.size[0]
height = baseImage.size[1]

# Build a base image with a solid background color.
ancestorImage = np.zeros((height, width, 3), np.uint8)
ancestorImage[:] = backgroundColor

showImage(ancestorImage) # Preview base image

### Create gene class

In [ ]:
class gene: # Defines a single gene, which describes how to draw a single circle.
    def __init__(self, maximumRadius, minimumRadius, height, width, numColors): # Basic constructor
        self.maximumRadius = maximumRadius
        self.minimumRadius = minimumRadius
        self.height = height
        self.width = width
        self.numColors = numColors
        self.radius = 0
        self.center = (0, 0)
        self.color = 0
        self.alpha = 0
        self.completelyRandomize() # Randomize values at initialization.
    
    def revert(self): # Reverts circle to previous properties (in case a mutation was unhelpful.)
        self.radius = self.history[0]
        self.center = self.history[1]
        self.color = self.history[2]
        self.alpha = self.history[3]
    
    def completelyRandomize(self): # Completely randomizes the values 
        self.radius = randint(self.minimumRadius, self.maximumRadius)
        self.center = (randint(0 - int(self.radius/5), self.width + int(self.radius/5)), randint(0 - int(self.radius/5), self.height + int(self.radius/5)))
        self.color = randint(0, self.numColors-2)
        self.alpha = random() * 0.25 + 0.1

    def mutate(self, complete=False): # Mutates a circle randomly.  If complete is True, this will always perform a complete mutation.
        self.history = [self.radius, self.center, self.color, self.alpha]
        seed = random()
        if seed < 0.5: # 50% chance of complete mutation, 50% chance of just changing the radius of the circle
            self.completelyRandomize()
        else:
            self.radius = randint(self.minimumRadius, self.maximumRadius)

### Evolve


In [ ]:
topScore = inf # original best score is infinite, so the first run will always overwrite it.
topImage = None # the best image so far.

sequence = [gene(MAXIMUM_RADIUS, MINIMUM_RADIUS, height, width, NUM_COLORS) for _ in range(NUM_CIRCLES)] # Generate initial gene sequence.

changes = 0 # Record number of total changes made. (some generations may have failed mutations that do not affect the sequence.)

for generationIndex in range(NUM_GENERATIONS): # loop through each generation

    # grab the gene at the bottom of the sequence (which corresponds to the circle at the very bottom of the image, depthwise)
    mutatedGene = sequence[0]
    # mutate it
    mutatedGene.mutate()

    image = ancestorImage.copy() # Create a new copy of the parent image.

    # delete the gene
    del sequence[0]

    # add the rest of the circles normally
    for gene in sequence[1:]:
        addCircle(image, gene)

    # add the gene on top.
    addCircle(image, mutatedGene)

    # evaluate the mutation
    score = calculateError(image, BGRbaseImage)

    # if it was beneficial...
    if score < topScore:
        topScore = score
        topImage = image
        sequence.append(mutatedGene) # Place the gene on top of the sequence again.
        changes += 1 # record a change
    else:
        mutatedGene.revert() # oopsies! revert changes.
        sequence.insert(0, mutatedGene) # insert gene back into the end of the sequence so it can be mutated again next generation.

    # Periodic checks on progress, every 100 generations.
    if generationIndex % 100 == 0:
        print(f"generation:{generationIndex}")
        print(f"number of changes:{changes}")
        showImage(topImage)
